In [4]:
import gurobipy as gp
from gurobipy import GRB
from itertools import product
from math import sqrt
import numpy as np
import random as rd
import copy

In [5]:
def read_data(file_name):
    edge = []
    with open(file_name) as f:
        data = f.readlines()
    _,p,v = data[0].replace('\n','').split(' ')
    for i in data[1:]:
        if i == '\n':
            break
        line_data = i.split(' ')
        edge.append((int(line_data[1]),int((line_data[2]).replace('\n', ''))))
    return edge,int(p),int(v)

In [6]:
edge,p,v = read_data('text.txt')

In [38]:
m = gp.Model('max-cut')

In [46]:
x = m.addVars(p,vtype=GRB.BINARY,name='x')
e = m.addVars(len(edge),vtype=GRB.BINARY,name='edge')
m.addConstrs(e[j] <= x[i[0]-1] + x[i[1]-1] for j,i in enumerate(edge))
m.addConstrs(e[j] <= 2 -x[i[0]-1] - x[i[1]-1] for j,i in enumerate(edge))
m.setObjective(gp.quicksum(e[i] for i in range(len(e))),GRB.MAXIMIZE)
m.optimize()

Gurobi Optimizer version 9.0.2 build v9.0.2rc0 (win64)
Optimize a model with 7342 rows, 6027 columns and 22026 nonzeros
Model fingerprint: 0x975fc689
Variable types: 0 continuous, 6027 integer (6027 binary)
Coefficient statistics:
  Matrix range     [1e+00, 1e+00]
  Objective range  [1e+00, 1e+00]
  Bounds range     [1e+00, 1e+00]
  RHS range        [2e+00, 2e+00]

MIP start from previous solve produced solution with objective 482 (0.06s)
MIP start from previous solve produced solution with objective 486 (5.87s)
Loaded MIP start from previous solve with objective 486
Processed MIP start in 12.29 seconds

Presolve removed 5870 rows and 5166 columns
Presolve time: 0.01s
Presolved: 1472 rows, 861 columns, 4416 nonzeros
Variable types: 0 continuous, 861 integer (861 binary)

Root simplex log...

Iteration    Objective       Primal Inf.    Dual Inf.      Time
       0    7.3600000e+02   7.360000e+02   0.000000e+00     12s
     132    7.3600000e+02   0.000000e+00   0.000000e+00     12s

Root

  3796  1368     cutoff   27       516.00000  531.28215  2.96%  1211  445s
  3872  1415     cutoff   30       516.00000  531.27624  2.96%  1228  462s
  3981  1473     cutoff   26       516.00000  531.23025  2.95%  1236  478s
  4065  1541  524.09693   23  954  516.00000  531.23025  2.95%  1252  494s
  4187  1570  518.25973   27  940  516.00000  531.16436  2.94%  1266  510s
  4310  1579  519.90669   25  948  516.00000  531.12166  2.93%  1272  528s
  4450  1605  526.65138   21  966  516.00000  530.81762  2.87%  1280  546s
  4575  1660  524.07766   24  953  516.00000  530.55399  2.82%  1285  566s
  4732  1695     cutoff   27       516.00000  530.37611  2.79%  1289  585s
  4867  1734  525.02943   21  960  516.00000  530.27217  2.77%  1293  607s
  5033  1790     cutoff   25       516.00000  530.27217  2.77%  1298  629s
  5169  1845     cutoff   26       516.00000  530.27217  2.77%  1311  651s
  5294  1884  518.90759   24  946  516.00000  530.27217  2.77%  1324  674s
  5471  1942  517.72726  

In [7]:
def ini_solution(p):
    """
    node's number
    """
    solution = []
    for i in range(p):
        solution.append(rd.randint(0,1))
    return solution

In [8]:
solution = ini_solution(p)

In [9]:
def get_obj(solution):
    obj = 0
    for i in range(len(solution)):
        for a,b in edge:
            if a == i:
                if solution[b] != solution[i]:
                    obj += 1
    return obj

In [10]:
print(get_obj(solution))

359


In [106]:
def ls(solution):
    for i in range(1000):
        new_solution = copy.deepcopy(solution)
        j = rd.randint(0,len(solution)-1)
        obj = get_obj(new_solution)
        if new_solution[j]==1:
            new_solution[j]=0
        else:
            new_solution[j]=1
        new_obj = get_obj(new_solution)
        if new_obj>obj:
            solution = new_solution
            print(new_obj)
    return solution

In [43]:
class ga_solution:
    obj = 0
    solution = []
    def __init__(self,p):
        self.solution = ini_solution(p)
        self.obj = get_obj(self.solution)
    def mutation(self):
        j = rd.randint(0, len(self.solution)-1)
        if self.solution[j] == 1:
            self.solution[j] = 0
        else:
            self.solution[j] = 1
        self.obj = get_obj(self.solution)
    def update(self):
        self.obj = get_obj(self.solution)

In [44]:
solution = ga_solution(p)
print(solution.obj)

378


In [60]:
def crossover(solutions):
    p = rd.randint(0, len(solutions)-1)
    q = rd.randint(0, len(solutions)-1)
    i = rd.randint(0,len(solutions[p].solution)-1)
    j = rd.randint(0,len(solutions[p].solution)-1)
    if i > j:
        i,j = j,i
    mid = solutions[p].solution[i:j]
    solutions[p].solution[i:j] = solutions[q].solution[i:j]
    solutions[q].solution[i:j] = mid
    solutions[p].update()
    solutions[q].update()
    
    return solutions
    

    

In [61]:
def select(solutions):
    """
    轮盘赌算法
    """
    new_solutions = []
    obj_sum = sum([i.obj for i in solutions])
    perc = [i.obj/obj_sum for i in solutions]
    for i in range(len(solutions)-1):
        rn = rd.random()
        for j in range(len(perc)-1):
            if rn >= sum(perc[:j]):
                new_solutions.append(solutions[j])
    return new_solutions
    

In [66]:
def ga(solutions):
    for i in range(100):
        solutions = crossover(solutions)
        solutions = select(solutions)
        for j in range(10):
            t = rd.randint(0,len(solutions)-1)
            print(solutions[t].obj)
            solutions[t].mutation()
    p_mean = sum(i.obj for i in solutions)/len(solutions)
    print(p_mean)
    
        

In [ ]:
solutions = []
for i in range(20):
    solution = ga_solution(p)
    solutions.append(solution)

ga(solutions)

379
366
362
368
359
359
367
350
375
364
363
369
370
369
353
375
377
379
370
372
